# Write Experiments

This file is used to generate JSON files for running experients

JSON files will be written for main.py to read arguments from, rather than going from the command-line

In [9]:
import pickle
import pandas as pd
from datetime import datetime

In [2]:
config = {
    'exp_name': 'hello_world',
    'save_name': 'hello_world',
    'num_env_steps': 10000,
    'env_name': 'CartPole-v0',
    'recurrent_policy': True,
    'algo': 'ppo',
    'num_mini_batch': 4,
    'num_processes': 4
}
pickle.dump(config, open('experiment_configs/hello_world', 'wb'))

## Scheduler functions

These functions are found in scheduler.py (testing them here), used to run and track the experiments in experiment_configs

In [2]:
from scheduler import *

In [3]:
run_experiment('hello_world')